## 4.1 이미지 다루기

### 4.1.1 컬러 채널 더하기

### 4.1.2 이미지 파일 로딩

In [2]:
import imageio

In [3]:
img_arr = imageio.imread('./bobby.jpg')
img_arr.shape

/tmp/ipykernel_20035/2865455528.py:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread('./bobby.jpg')


(720, 1280, 3)

### 4.1.3 레이아웃 변경하기

In [4]:
import torch

In [5]:
img = torch.from_numpy(img_arr)
out = img.permute(2, 0, 1)
out.shape

torch.Size([3, 720, 1280])

In [6]:
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)

In [7]:
import os

In [8]:
data_dir = os.getcwd()
data_dir

'/data/heerak/workspace/pytorch_dl_master/p1ch4'

In [11]:
filenames = [name for name in os.listdir(data_dir)
             if 'cat' in os.path.splitext(name)[0] and os.path.splitext(name)[-1] == '.png']
filenames

['cat2.png', 'cat1.png', 'cat3.png']

In [12]:
for i, filename in enumerate(filenames):
    img_arr = imageio.imread(os.path.join(data_dir, filename))
    img_t = torch.tensor(img_arr)
    img_t = img_t.permute(2, 0, 1)
    img_t = img_t[:3]  # 투명도 알파 채널까지 있는 이미지도 있지만, RGB만 사용하므로 첫 세 개 채널만 유지
    batch[i] = img_t

/tmp/ipykernel_20035/2508321617.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread(os.path.join(data_dir, filename))


In [13]:
batch

tensor([[[[202, 193, 190,  ...,  13,  13,  12],
          [199, 192, 189,  ...,  14,  14,  14],
          [198, 193, 188,  ...,  12,  12,  12],
          ...,
          [ 93,  82,  76,  ...,  36,  36,  36],
          [ 75,  68, 101,  ...,  36,  36,  37],
          [ 85, 103,  90,  ...,  36,  37,  38]],

         [[151, 139, 133,  ...,   9,   9,   8],
          [151, 140, 134,  ...,  11,  11,  11],
          [152, 143, 134,  ...,  11,  11,  11],
          ...,
          [ 57,  45,  39,  ...,  26,  26,  26],
          [ 33,  26,  59,  ...,  26,  26,  27],
          [ 40,  58,  45,  ...,  26,  27,  28]],

         [[ 68,  53,  44,  ...,   6,   6,   5],
          [ 67,  54,  44,  ...,   6,   6,   6],
          [ 67,  56,  44,  ...,   6,   6,   6],
          ...,
          [ 31,  19,  12,  ...,  17,  17,  17],
          [ 11,   2,  35,  ...,  17,  17,  18],
          [ 19,  37,  22,  ...,  17,  18,  19]]],


        [[[156, 152, 124,  ..., 150, 149, 158],
          [174, 134, 165,  ..., 120

### 4.1.4 데이터 정규화

In [14]:
batch = batch.float()
batch /= 255.0
batch

tensor([[[[0.7922, 0.7569, 0.7451,  ..., 0.0510, 0.0510, 0.0471],
          [0.7804, 0.7529, 0.7412,  ..., 0.0549, 0.0549, 0.0549],
          [0.7765, 0.7569, 0.7373,  ..., 0.0471, 0.0471, 0.0471],
          ...,
          [0.3647, 0.3216, 0.2980,  ..., 0.1412, 0.1412, 0.1412],
          [0.2941, 0.2667, 0.3961,  ..., 0.1412, 0.1412, 0.1451],
          [0.3333, 0.4039, 0.3529,  ..., 0.1412, 0.1451, 0.1490]],

         [[0.5922, 0.5451, 0.5216,  ..., 0.0353, 0.0353, 0.0314],
          [0.5922, 0.5490, 0.5255,  ..., 0.0431, 0.0431, 0.0431],
          [0.5961, 0.5608, 0.5255,  ..., 0.0431, 0.0431, 0.0431],
          ...,
          [0.2235, 0.1765, 0.1529,  ..., 0.1020, 0.1020, 0.1020],
          [0.1294, 0.1020, 0.2314,  ..., 0.1020, 0.1020, 0.1059],
          [0.1569, 0.2275, 0.1765,  ..., 0.1020, 0.1059, 0.1098]],

         [[0.2667, 0.2078, 0.1725,  ..., 0.0235, 0.0235, 0.0196],
          [0.2627, 0.2118, 0.1725,  ..., 0.0235, 0.0235, 0.0235],
          [0.2627, 0.2196, 0.1725,  ..., 0

In [15]:
n_channels = batch.shape[1]
for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean) / std

In [16]:
batch

tensor([[[[ 0.9595,  0.7999,  0.7467,  ..., -2.3915, -2.3915, -2.4092],
          [ 0.9063,  0.7822,  0.7290,  ..., -2.3738, -2.3738, -2.3738],
          [ 0.8886,  0.7999,  0.7113,  ..., -2.4092, -2.4092, -2.4092],
          ...,
          [-0.9731, -1.1681, -1.2745,  ..., -1.9837, -1.9837, -1.9837],
          [-1.2922, -1.4163, -0.8312,  ..., -1.9837, -1.9837, -1.9660],
          [-1.1149, -0.7958, -1.0263,  ..., -1.9837, -1.9660, -1.9482]],

         [[ 0.6908,  0.4632,  0.3494,  ..., -2.0024, -2.0024, -2.0214],
          [ 0.6908,  0.4822,  0.3684,  ..., -1.9645, -1.9645, -1.9645],
          [ 0.7098,  0.5391,  0.3684,  ..., -1.9645, -1.9645, -1.9645],
          ...,
          [-1.0920, -1.3196, -1.4334,  ..., -1.6800, -1.6800, -1.6800],
          [-1.5472, -1.6800, -1.0541,  ..., -1.6800, -1.6800, -1.6610],
          [-1.4144, -1.0730, -1.3196,  ..., -1.6800, -1.6610, -1.6420]],

         [[-0.4598, -0.7644, -0.9472,  ..., -1.7190, -1.7190, -1.7394],
          [-0.4801, -0.7441, -